In [1]:
#of big help https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059
#end up by not using BeautifulSoup
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#Store the contents of the website under doc
doc = lh.fromstring(source.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')
len(tr_elements)

294

In [2]:
#Check the length of the first 10 rows: the lenght was 3
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [3]:
#let’s parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
   # print ('%d:"%s"'%(i,name))
    col.append((name,[]))
  #  print (col)

In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [5]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [6]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [7]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [8]:
df.shape
df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [9]:
#the last column name has an extra \n to the end. Let's remove it
df.rename(columns={'Neighbourhood\n':'Neighbourhood', 'Postcode': 'PostalCode'}, inplace=True)

#remove Borough with 'not assigned' value
df_clean= df[df['Borough'] != 'Not assigned']
df_clean.reset_index(inplace=True)

df_clean=df_clean[['PostalCode', 'Borough', 'Neighbourhood']]

df_clean.head()


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M5A,Downtown Toronto,Regent Park\n
4,M6A,North York,Lawrence Heights\n


In [10]:
#getting rid of a stuborn '\n' at teh end of my Neighbourhood values.

df_clean['Neighbourhood'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
df_clean.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
#replace 'Not assigned' neighbourhooh with its Borough value.
df_clean['Neighbourhood'].replace('Not assigned',df_clean['Borough'], inplace=True)
df_clean.head(7)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park


In [12]:
df_clean.shape

(211, 3)

In [13]:
df_grouped = df_clean.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [14]:
#Print number of rows in the dataframe
df_grouped.shape

(103, 3)

<h2>Begining of the geocoder section</h2>

In [26]:
df_geo = pd.read_csv('geoSpatialDatacanada.csv')

In [27]:
df_geo.head()
#df_geo.shape

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
#combine the two dataframes into one
 df_toronto = pd.concat([df_grouped, df_geo[['Latitude', 'Longitude']]], axis=1)

In [32]:
df_toronto.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
